- onednn_verbose format
```
{
  'engine': 'cpu'   
  'prim_kind': 'convolution'   
  'impl': 'brgconv:avx512_core'       
  'prop_kind': 'forward_training'   
  'mds': [
    {'arg': 'src'   'data_type': 'f32'   'properties': ''   'format_kind': 'blocked'   'tag': 'acdb'   'strides': ''   'flags': {'value': 'f0'}}   
    {'arg': 'wei'   'data_type': 'f32'            'properties': ''    'format_kind': 'blocked'   'tag': 'Acdb32a'   'strides': ''   'flags': {'value': 'f0'}}    
    {'arg': 'bia'                         'data_type': 'f32'   'properties': ''          'format_kind': 'blocked'   'tag': 'a'   'strides': ''   'flags': {'value': 'f0'}}   
    {'arg': 'dst'   'data_type': 'f32'   'properties': ''   'format_kind': 'blocked'   'tag': 'acdb'   'strides': ''   'flags': {'value': 'f0'}}
  ]   
  'exts': {
    'attr-post-ops': 
      [{'alg': 'eltwise_relu'   'alpha': '1.0'     'beta': '0.0'      'scale': '1.0'}]
      'attr-scratchpad': 'user'
  }   
  'aux': {'alg': 'convolution_direct'}   
  'shapes': 'mb256_ic256oc512_ih14oh7kh3sh2dh0ph1_iw14ow7kw3sw2dw0pw1'      
  'time': '11.6699\n'
}
```  

In [13]:
!ONEDNN_VERBOSE=1 python3 -W ignore chap05.py --data=/home/taosy/datasets/resnet50 --evaluate --pretrained --profile > /tmp/onednn-1.log  2>/dev/null
!tail -n 20 /tmp/onednn-1.log
!echo =============================================================
!echo
!/home/taosy/repo/github/oneDNN/scripts/verbose_converter/verbose_converter.py -i /tmp/onednn-1.log -g breakdown -k prim_kind  impl | column -t -s,

Iteration: 19, inference time: 0.9458277225494385 sec.
 *   Acc@1 76.660 Acc@5 92.676
inference latency: 3.696078 ms
inference Throughput: 270.557056 images/s
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                               aten::mkldnn_convolution        53.74%        7.949s        53.80%        7.957s      19.892ms           400  
                                aten::native_batch_norm        17.35%        2.566s        17.42%        2.576s       6.441ms           400  
enumerate(DataLoader)#_MultiProcessingDataLoaderIter...        12.94%        1.914s        12.94%        1.914s      91.151ms      

In [14]:
!ONEDNN_VERBOSE=1 python3 -W ignore chap05.py --data=/home/taosy/datasets/resnet50 --evaluate --pretrained --ipex --profile > /tmp/onednn-2.log  2>/dev/null
!tail -n 20 /tmp/onednn-2.log
!echo =============================================================
!echo
!/home/taosy/repo/github/oneDNN/scripts/verbose_converter/verbose_converter.py -i /tmp/onednn-2.log -g breakdown -k prim_kind  impl | column -t -s,

Iteration: 19, inference time: 0.7473795413970947 sec.
 *   Acc@1 76.660 Acc@5 92.676
inference latency: 3.061970 ms
inference Throughput: 326.587150 images/s
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                   torch_ipex::convolution_forward_impl        64.56%        8.608s        66.28%        8.838s      22.096ms           400  
enumerate(DataLoader)#_MultiProcessingDataLoaderIter...        14.68%        1.958s        14.69%        1.959s      93.263ms            21  
                                              aten::add         6.60%     879.952ms         6.60%     879.962ms       5.432ms      

In [15]:
!ONEDNN_VERBOSE=1 python3 -W ignore chap05.py --data=/home/taosy/datasets/resnet50 --evaluate --pretrained --jit --profile > /tmp/onednn-3.log  2>/dev/null
!tail -n 20 /tmp/onednn-3.log
!echo =============================================================
!echo
!/home/taosy/repo/github/oneDNN/scripts/verbose_converter/verbose_converter.py -i /tmp/onednn-3.log -g breakdown -k prim_kind  impl | column -t -s,

Iteration: 19, inference time: 0.574678897857666 sec.
 *   Acc@1 76.660 Acc@5 92.676
inference latency: 2.139743 ms
inference Throughput: 467.345930 images/s
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     ipex_prepack::convolution_relu_run        32.10%        4.316s        33.66%        4.526s      26.469ms           171  
                 ipex_prepack::convolution_add_relu_run        23.33%        3.137s        23.33%        3.137s      20.639ms           152  
                                                forward        16.61%        2.234s        84.00%       11.294s     564.695ms       